# Criação, treino e teste do Modelo

## Instalando bibliotecas 

In [1]:
!pip install tensorflow


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Leitura e Verificação dos Dados

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [8]:
df = pd.read_parquet('DF_KNRS_COMUM_PROCESSADO.parquet')
print(df['TEM_FALHA_ROD'].value_counts())  # Verificando a distribuição da variável alvo
print(df.isnull().sum())
print('\n----------\n')

df = df.dropna()  # Removendo valores nulos
df = df.drop(columns=["QTD_HALLE_ROD", "QTD_HALLE_AGUA", "QTD_HALLE_ZP8", "QTD_HALLE_ZP8R"])

print('\n----------\n')
print(df['TEM_FALHA_ROD'].value_counts())  # Verificando a distribuição da variável alvo
print(df.isnull().sum())

TEM_FALHA_ROD
0.0    64841
1.0     4653
Name: count, dtype: int64
KNR                           0
QTD_STATUS_1_OK               0
QTD_STATUS_1_NOK              0
QTD_STATUS_2_OK               0
QTD_STATUS_2_NOK              0
QTD_STATUS_718_OK             0
QTD_STATUS_718_NOK            0
TEMPO_MEDIO                6253
MOTOR                     12675
COR                       12675
QTD_HALLE_                12677
QTD_HALLE_AGUA            12677
QTD_HALLE_BUY             12677
QTD_HALLE_CAB             12677
QTD_HALLE_DKA             12677
QTD_HALLE_ESPC            12677
QTD_HALLE_PROC            12677
QTD_HALLE_PROF            12677
QTD_HALLE_PVC             12677
QTD_HALLE_ROD             12677
QTD_HALLE_RUID            12677
QTD_HALLE_TLUI            12677
QTD_HALLE_ZP5             12677
QTD_HALLE_ZP5A            12677
QTD_HALLE_ZP6             12677
QTD_HALLE_ZP61            12677
QTD_HALLE_ZP62            12677
QTD_HALLE_ZP7             12677
QTD_HALLE_ZP8             12677
QTD_HA

In [9]:
qtd_ocorrencias = df[df['QTD_HALLE_ESPC'] > 1].shape[0]
print(f"Ocorrências de valor 1 na coluna QTD_HALLE_ESPC: {qtd_ocorrencias}")


Ocorrências de valor 1 na coluna QTD_HALLE_ESPC: 1409


In [10]:
df.head()

,KNR,QTD_STATUS_1_OK,QTD_STATUS_1_NOK,QTD_STATUS_2_OK,QTD_STATUS_2_NOK,QTD_STATUS_718_OK,QTD_STATUS_718_NOK,TEMPO_MEDIO,MOTOR,COR,...,QTD_SGROUP_-2,QTD_SGROUP_1,QTD_SGROUP_133,QTD_SGROUP_137,QTD_SGROUP_140,QTD_SGROUP_2,QTD_SGROUP_4,QTD_SGROUP_5,QTD_SGROUP_9830946,TEM_FALHA_ROD
0,2023-5076008,0,0,0,0,1,2,54.250000,DHS,0Q0Q,...,1.0,16.0,0.0,0.0,1.0,8.0,3.0,4.0,0.0,1.0
1,2023-5076015,0,0,0,0,1,1,54.250000,DHS,0Q0Q,...,1.0,5.0,0.0,0.0,1.0,4.0,4.0,2.0,0.0,0.0
2,2024-0516009,0,0,16,0,40,0,54.250000,CWL,5T5T,...,1.0,5.0,0.0,0.0,0.0,5.0,2.0,6.0,3.0,0.0
3,2024-0526019,0,0,0,0,1,0,26.416667,DHS,0Q0Q,...,0.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0
4,2024-0526096,0,0,4,0,53,1,26.416667,DHS,K2K2,...,0.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0


## Pré-processamento dos dados

In [46]:
# Criando variáveis dummies
df = pd.get_dummies(df, columns=['COR', 'MOTOR'], drop_first=True)

# Removendo colunas desnecessárias
df = df.drop(columns=["KNR"])

## Balanceamento das Classes

In [47]:
from imblearn.under_sampling import RandomUnderSampler  # Importa o método RandomUnderSampler para balanceamento de classes

rus = RandomUnderSampler(random_state=0)  # Inicializa o RandomUnderSampler com uma semente de aleatoriedade fixa

X = df.drop(columns=['TEM_FALHA_ROD'])  # Removendo a variável alvo, mantendo apenas as variáveis preditoras
y = df['TEM_FALHA_ROD']  # Definindo a variável alvo

X_resampled, y_resampled = rus.fit_resample(X, y)  # Aplica o balanceamento de classes aos dados, retornando as amostras balanceadas

## Dividindo o dataset entre treino e teste

In [48]:
from sklearn.model_selection import train_test_split  # Importa a função para dividir os dados em conjuntos de treino e teste

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)  
# Divide os dados balanceados em conjuntos de treino (80%) e teste (20%), com uma semente de aleatoriedade fixa para reprodução

## Redimensionamento dos dados

In [49]:
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))  
# Altera a forma de X_train para (n amostras, 1, n características) para compatibilidade com redes neurais que esperam uma dimensão adicional

X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))  
# Altera a forma de X_test de maneira semelhante

X_train = np.array(X_train, dtype=np.float32)  
# Converte X_train para um array NumPy com tipo de dado float32

y_train = np.array(y_train, dtype=np.float32)  
# Converte y_train para um array NumPy com tipo de dado float32

## Construção dos Modelos (LSTM e GRU)

### Modelo GRU

In [50]:
model_gru = Sequential()  # Cria um modelo sequencial, que é uma pilha linear de camadas

model_gru.add(GRU(50, input_shape=(X_train.shape[1], X_train.shape[2])))  
# Adiciona uma camada GRU com 50 unidades. A forma de entrada é (n timesteps, n características)

model_gru.add(Dropout(0.2))  
# Adiciona uma camada Dropout para prevenir overfitting, desativando aleatoriamente 20% dos neurônios durante o treinamento

model_gru.add(Dense(1, activation='sigmoid'))  
# Adiciona uma camada densa de saída com 1 neurônio e ativação sigmoide para uma tarefa de classificação binária

model_gru.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])  
# Compila o modelo usando o otimizador Adam com taxa de aprendizado de 0.001, a função de perda de entropia cruzada binária e a métrica de acurácia

C:\Users\Inteli\Documents\GitHub\2024-2A-T08-EC07-G05\src\notebooks\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [51]:
# Treinando o modelo GRU
history_gru = model_gru.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5030 - loss: 0.7710 - val_accuracy: 0.4715 - val_loss: 0.7075
Epoch 2/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4978 - loss: 0.7282 - val_accuracy: 0.4632 - val_loss: 0.7080
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5014 - loss: 0.7151 - val_accuracy: 0.5045 - val_loss: 0.6941
Epoch 4/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5172 - loss: 0.7008 - val_accuracy: 0.5060 - val_loss: 0.6965
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5020 - loss: 0.7085 - val_accuracy: 0.4947 - val_loss: 0.6967
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5126 - loss: 0.7015 - val_accuracy: 0.5113 - val_loss: 0.6942
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5147 - loss: 0.6991 - val_accuracy: 0.5068 - val_loss: 0.6920
Epoch 8/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5042 - loss: 0.7011 - val_accu

### Modelo LSTM

In [36]:
# Definir o modelo LSTM
model_lstm = Sequential()  # Cria um modelo sequencial para o LSTM

# Camada LSTM
model_lstm.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))  
# Adiciona uma camada LSTM com 50 unidades. A forma de entrada é (n timesteps, n características)

model_lstm.add(Dropout(0.2))  
# Adiciona uma camada Dropout para prevenir overfitting, desativando aleatoriamente 20% dos neurônios durante o treinamento

# Camada de saída
model_lstm.add(Dense(1, activation='sigmoid'))  
# Adiciona uma camada densa de saída com 1 neurônio e ativação sigmoide para uma tarefa de classificação binária

# Compilar o modelo
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])  
# Compila o modelo usando o otimizador Adam com taxa de aprendizado de 0.001, a função de perda de entropia cruzada binária e a métrica de acurácia

# Resumo do modelo
model_lstm.summary()  # Exibe um resumo da arquitetura do modelo

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 50)                  │          20,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,651 (80.67 KB)

 Trainable params: 20,651 (80.67 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Treinando o modelo LSTM
history_lstm = model_lstm.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)

Epoch 1/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.5150 - loss: 0.7090 - val_accuracy: 0.5008 - val_loss: 0.6931
Epoch 2/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5081 - loss: 0.7012 - val_accuracy: 0.4827 - val_loss: 0.6959
Epoch 3/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5204 - loss: 0.6985 - val_accuracy: 0.5075 - val_loss: 0.6928
Epoch 4/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5049 - loss: 0.7001 - val_accuracy: 0.5158 - val_loss: 0.6929
Epoch 5/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5380 - loss: 0.6912 - val_accuracy: 0.5375 - val_loss: 0.6903
Epoch 6/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5195 - loss: 0.6974 - val_accuracy: 0.5480 - val_loss: 0.6892
Epoch 7/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5345 - loss: 0.6922 - val_accuracy: 0.5008 - val_loss: 0.6934
Epoch 8/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5188 - loss: 0.6951 - val_accu

## Avaliando o Modelo

### GRU

In [52]:
# Converter X_test e y_test para float32
X_test = np.array(X_test).astype('float32')
y_test = np.array(y_test).astype('float32')

# Avaliar o modelo GRU com os dados de teste
loss_gru, accuracy_gru = model_gru.evaluate(X_test, y_test)
print(f'Test Loss GRU: {loss_gru}')
print(f'Test Accuracy GRU: {accuracy_gru}')

# Fazer previsões com o modelo GRU
y_pred_gru = model_gru.predict(X_test)
y_pred_gru = (y_pred_gru > 0.5).astype(int)

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred_gru))


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5182 - loss: 0.6923
Test Loss GRU: 0.6931670904159546
Test Accuracy GRU: 0.5369369387626648
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
              precision    recall  f1-score   support

         0.0       0.54      0.63      0.58       841
         1.0       0.54      0.44      0.48       824

    accuracy                           0.54      1665
   macro avg       0.54      0.54      0.53      1665
weighted avg       0.54      0.54      0.53      1665



### LSTM

In [34]:
# Avaliar o modelo LSTM com os dados de teste
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test, y_test)
print(f'Test Loss LSTM: {loss_lstm}')
print(f'Test Accuracy LSTM: {accuracy_lstm}')

# Fazer previsões com o modelo LSTM
y_pred_lstm = model_lstm.predict(X_test)
y_pred_lstm = (y_pred_lstm > 0.5).astype(int)

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred_lstm))

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - accuracy: 0.9537 - loss: 0.1107
Test Loss LSTM: 0.09792882204055786
Test Accuracy LSTM: 0.9615615606307983
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96       841
         1.0       1.00      0.92      0.96       824

    accuracy                           0.96      1665
   macro avg       0.96      0.96      0.96      1665
weighted avg       0.96      0.96      0.96      1665

